In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import cv2
import numpy as np
from torchvision import models


class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.activations = []
        self.gradients = []

    def _save_gradient(self, grad):
        self.gradients.append(grad)

    def _get_activations_gradient(self):
        grads = self.gradients[-1]
        activations = self.activations[-1]
        return grads, activations

    def _get_linear_weights(self, grads):
        return torch.mean(grads, axis=(2, 3))

    def _get_conv_weights(self, grads):
        return torch.mean(grads, axis=(2, 3))

    def _get_gradcam_image(self, conv_output, grads):
        weights = self._get_conv_weights(grads)[0]
        cam = torch.zeros(conv_output.shape[1:], dtype=torch.float32, device=conv_output.device)
        for i, w in enumerate(weights):
            cam += w * conv_output[i, :, :]
        cam = F.relu(cam)
        cam = F.interpolate(cam.unsqueeze(0).unsqueeze(0), size=(224, 224), mode='bilinear', align_corners=False)
        return cam.squeeze().cpu().numpy()

    def __call__(self, x, class_idx=None, device='cpu', height=None, width=None):
        x = x.to(device)
        self.activations = []
        self.gradients = []
        output = x
        for name, module in self.model.named_children():
            if name == self.target_layer:
                output.register_hook(self._save_gradient)
                self.activations.append(output)
                output = output.detach()
            elif isinstance(module, torch.nn.Sequential) or isinstance(module, torch.nn.ModuleList):
                for _, module in module.named_children():
                    output = module(output)
                    if isinstance(module, torch.nn.ReLU):
                        output.register_hook(self._save_gradient)
                        self.activations.append(output)
                        output = output.detach()
            else:
                output = module(output)
        if class_idx is None:
            class_idx = torch.argmax(output)
        self.logits = output[:, class_idx]
        self.logits.backward(retain_graph=True)
        grads, activations = self._get_activations_gradient()
        conv_output = activations.detach()
        cam = self._get_gradcam_image(conv_output, grads)
        upsampled_cam = cv2.resize(cam, (width, height))
        return upsampled_cam



    
        
      


In [ ]:
import torch
from torchvision import models
from efficientnet_pytorch import EfficientNet

# Cargar los pesos de EfficientNetB2
model_weights = torch.load("/Users/guill/Escritorio/CNN/best_model_EfficientNetB2_FrozenLast8.pth")

# Construir el modelo EfficientNetB2 con los pesos cargados
model = EfficientNet.from_pretrained('efficientnet-b2', weights_path=None)
model.load_state_dict(model_weights)

# Establecer el modelo en modo de evaluación
model.eval()


In [ ]:
model = models.resnet18(pretrained=True)
gradcam = GradCAM(model, target_layer='layer4')
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
img = cv2.imread("/Users/guill/Escritorio/TesisLisboa/archive/DFU/TestSet/test/Abnormal (63).jpg")

resized_img = cv2.resize(img, (224, 224))
resized_part = torch.from_numpy(resized_img).permute(2, 0, 1).unsqueeze(0).float().div(255)
cam = gradcam(resized_part.to(device), class_idx=0, device=device, height=img.shape[0], width=img.shape[1])
heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)
result = heatmap * 0.3 + image.squeeze().permute(1, 2, 0).cpu().numpy() * 0.5
result = result / np.max(result)
cv2.imshow('Result', result)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np
import torch
from torchvision import transforms
import albumentations as A
from albumentations.pytorch import ToTensorV2
# Carga la imagen
image = cv2.imread("/Users/guill/Escritorio/TesisLisboa/archive/DFU/TestSet/test/Abnormal (63).jpg")

# Divide la imagen en partes
num_rows = 3
num_cols = 3
height, width = image.shape[:2]
part_height = height // num_rows
part_width = width // num_cols


In [ ]:
params = {
    
    #otros modelos:"resnet18""alexnet""vgg16""densenet121" ResNet18, ResNet34, ResNet50, ResNet101, ResNet152
#VGG11, VGG13, VGG16, VGG19
#DenseNet121, DenseNet161, DenseNet169, DenseNet201
#InceptionV3, InceptionResNetV2
#MobileNetV2, MobileNetV3
#EfficientNetB0, EfficientNetB1, EfficientNetB2, EfficientNetB3, EfficientNetB4, EfficientNetB5, EfficientNetB6, EfficientNetB7
    "model": "densenet121",
    "device": "cpu",  # usar 'cuda' para usar la GPU
    "lr": 0.001, #cambiar a 0.01
    "batch_size": 64,#cambiar a 32
    "num_workers": 0,
    "epochs": 10,
}

In [ ]:
#EfficientNetB2
#congelo las ultimas 8 capa

import torch.nn as nn
from efficientnet_pytorch import EfficientNet

# Cargar la red EfficientNet pre-entrenada
model = EfficientNet.from_pretrained('efficientnet-b2')



# Congelar las primeras capas
for name, param in model.named_parameters():
    if not name.startswith('_conv_head.') and not name.startswith('_fc.') and not name.startswith('_bn1.') and not name.startswith('_blocks.22') and not name.startswith('_blocks.21') and not name.startswith('_blocks.20') and not name.startswith('_blocks.19') and not name.startswith('_blocks.18') and not name.startswith('_blocks.17') and not name.startswith('_blocks.16') and not name.startswith('_blocks.15'): 
        param.requires_grad = False


# Reemplazar el clasificador lineal con uno personalizado
num_ftrs = model._fc.in_features
model._fc = nn.Linear(num_ftrs, 1)

# Mover el modelo y el criterio al dispositivo especificado
model = model.to(params["device"])
criterion = nn.BCEWithLogitsLoss().to(params["device"])

# Definir el optimizador y ajustar solo los parámetros que no están congelados
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=params["lr"])

In [ ]:
# Carga el modelo

model.load_state_dict(torch.load("best_model_EfficientNetB2_FrozenLast8.pth"))
model.eval()

In [ ]:
# Crea una matriz para almacenar los mapas de activación GradCAM de cada parte
gradcam_matrix = np.zeros((num_rows, num_cols, part_height, part_width))


In [ ]:
test_transform = A.Compose(
    [
        A.SmallestMaxSize(max_size=160),
        A.CenterCrop(height=128, width=128),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(), #solo lo quito si en la funcion de arriba estoy ya creando el tensor
        
    ]
)


In [ ]:
# Utiliza GradCAM para obtener los mapas de activación de cada parte de la imagen
gradcam = GradCAM(model=model, target_layer=getattr(model._blocks[22], '_expand_conv'))
transform = test_transform
device = 'cuda' if torch.cuda.is_available() else 'cpu'
 
for i in range(num_rows):
    for j in range(num_cols):
        part = image[i*part_height:(i+1)*part_height, j*part_width:(j+1)*part_width]
        resized_part = cv2.resize(part, (224, 224))
        
        resized_part = test_transform(image=resized_part)['image']
        
        resized_part = resized_part.unsqueeze(0)
        resized_part.requires_grad_()
        with torch.no_grad():
            logits = model(resized_part)
        cam = gradcam(resized_part, device=device, height=part_height, width=part_width)
        cam.backward(retain_graph=True)

        upsampled_cam = cv2.resize(cam, (part_width, part_height))
        gradcam_matrix[i,j] = upsampled_cam


In [ ]:
# Une los mapas de activación GradCAM para crear una imagen completa de GradCAM
gradcam_image = np.zeros_like(image)
for i in range(num_rows):
    for j in range(num_cols):
        part = gradcam_matrix[i,j]
        x = j * part_width
        y = i * part_height
        gradcam_image[y:y+part_height, x:x+part_width] = part


In [ ]:
# Aplica la función de activación ReLU para obtener únicamente las partes activadas
heatmap = np.maximum(gradcam_image, 0)


In [ ]:
# Normaliza el mapa de calor entre 0 y 1
heatmap /= heatmap.max()


In [ ]:
# Crea una imagen en color a partir del mapa de calor y la imagen original
heatmap = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
output_image = heatmap * 0.5 + image * 0.5


In [ ]:
# Muestra la imagen resultante
cv2.imshow("GradCAM", output_image)
cv2.waitKey(0)